##Imports

In [41]:
import os
import torch
import torch.nn as nn
from torchtext.legacy import data, datasets 
from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# parameters
batch_size = 64
learning_rate = 0.001
training_epochs = 5

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cuda


## 1 Dataset

In [31]:
# IMDB Dataset

# data.Field 설명 #
# sequential인자 : TEXT는 Sequential 데이터라 True, Lable은 비Sequential이라 False로 설정
# batch_first : Batch를 우선시 하여, Tensor 크기를 (BATCH_SIZE, 문장의 최대 길이)로 설정
# lower : 소문자 전환 인자

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
trainset, validset, testset = datasets.SST.splits(TEXT, LABEL)

# data.Field.build_vocab() 라이브러리
# 문장 내 단어와 Integer index 를 매칭시키는 단어장(vocab)을 생성 == 워드 임베딩을 위한 Vocab 생성
# <UNK> = 0, <PAD> = 1 토큰도 추가.
# min_freq : 최소 5번 이상 등장한 단어들만 사전에 담겠다는 것. 
# 5번 미만으로 등장하는 단어는 UNK라는 토큰으로 대체

TEXT.build_vocab(trainset, min_freq=5)# TEXT 데이터를 기반으로 Vocab 생성
LABEL.build_vocab(trainset)# LABEL 데이터를 기반으로 Vocab 생성

# 학습용 데이터를 학습셋 80% 검증셋 20% 로 나누기
#trainset, validset = trainset.split(split_ratio=0.8)
# 매 배치마다 비슷한 길이에 맞춰 줄 수 있도록 iterator 정의
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, validset, testset), batch_size=batch_size,
        shuffle=True, repeat=False)

vocab_size = len(TEXT.vocab)
n_classes = 3 # Positive, Negative Class가 두 개

print("[TrainSet]: %d [ValSet]: %d [TestSet]: %d [Vocab]: %d [Classes] %d"
      % (len(trainset),len(validset), len(testset), vocab_size, n_classes))

[TrainSet]: 8544 [ValSet]: 1101 [TestSet]: 2210 [Vocab]: 3428 [Classes] 3


## 2 RNN model

+ Layer 설계 
  + Layer 1 
    + Embedding layer
    + Input size = n_Vocabs = 46159
    + Output size = Embedding size
  * Layer 2
    + GRU Layer
    + Input size = Embedding size
    + Output size = Hidden size
    + Dropout = 0.2
  * Layer 3
    + Linear Layer
    + Input size = Hidden size
    + Output size = n_Classes = 2




In [32]:
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        self.n_layers = n_layers # 일반적으로는 2

        #n_vocab : Vocab 안에 있는 단어의 개수, embed_dim : 임베딩 된 단어 텐서가 갖는 차원 값(dimension)
        self.embed = nn.Embedding(n_vocab, embed_dim)

        # hidden vector의 dimension과 dropout 정의
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)

        #앞에서 정의한 하이퍼 파라미터를 넣어 GRU 정의
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        
        #Input: GRU의 hidden vector(context), Output : Class probability vector
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        # Input data: 한 batch 내 모든 영화 평가 데이터
        
        x = self.embed(x)# 영화 평 임베딩
        x, _ = self.gru(x)  # [i, b, h] 출력값 :  (batch_size, 입력 x의 길이, hidden_dim)

        # h_t : Batch 내 모든 sequential hidden state vector의 제일 마지막 토큰을 내포한 (batch_size, 1, hidden_dim)형태의 텐서 추출
        # 다른 의미로 영화 리뷰 배열들을 압축한 hidden state vector
        h_t = x[:,-1,:]

        self.dropout(h_t)# dropout 설정 후, 

        # linear layer의 입력으로 주고, 각 클래스 별 결과 logit을 생성.
        out = self.out(h_t)  # [b, h] -> [b, o]
        return out

## 3 학습하기##

In [42]:
# contruct model
#from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train
corrects = 0
for batch in val_iter:
  x,y = batch.text.to(device), batch.label.to(device)
  y.data.sub_(1)
  hypothesis = model(x)
  corrects += (hypothesis.max(1)[1].view(y.size()).data == y.data).sum() 
  for epoch in range(training_epochs):
    avg_cost = 0
    for batch in train_iter:
      X, Y = batch.text.to(device), batch.label.to(device)
      Y.data.sub_(1)
      optimizer.zero_grad()
      hypothesis = model(X)
      cost = criterion(hypothesis, Y)
      cost.backward()
      optimizer.step()
      avg_cost += cost / batch_size
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    #print('batch =' , batch)
    print('accuracy = ', corrects/len(val_iter.dataset)*100.0)
    # model save
    torch.save(model.state_dict(), '/content/drive/MyDrive/model_before.pt')
  print('2 epoch 동안 accuracy 증가가 없어' ,epoch, 'epoch 학습 후 종료합니다')


 # final model save
torch.save(model.state_dict(), '/content/drive/MyDrive/model_final.pt')



[Epoch:    1] cost = 2.21398544
accuracy =  tensor(1.8165, device='cuda:0')
[Epoch:    2] cost = 2.1223073
accuracy =  tensor(1.8165, device='cuda:0')
[Epoch:    3] cost = 1.85662341
accuracy =  tensor(1.8165, device='cuda:0')
[Epoch:    4] cost = 1.58854115
accuracy =  tensor(1.8165, device='cuda:0')
[Epoch:    5] cost = 1.32199502
accuracy =  tensor(1.8165, device='cuda:0')
2 epoch 동안 accuracy 증가가 없어 4 epoch 학습 후 종료합니다
[Epoch:    1] cost = 1.01774716
accuracy =  tensor(4.9955, device='cuda:0')
[Epoch:    2] cost = 0.723435819
accuracy =  tensor(4.9955, device='cuda:0')
[Epoch:    3] cost = 0.474854439
accuracy =  tensor(4.9955, device='cuda:0')
[Epoch:    4] cost = 0.298289716
accuracy =  tensor(4.9955, device='cuda:0')
[Epoch:    5] cost = 0.172503293
accuracy =  tensor(4.9955, device='cuda:0')
2 epoch 동안 accuracy 증가가 없어 4 epoch 학습 후 종료합니다
[Epoch:    1] cost = 0.122922316
accuracy =  tensor(9.0827, device='cuda:0')
[Epoch:    2] cost = 0.106784448
accuracy =  tensor(9.0827, device='

In [43]:
# model load
model_new = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)
model_new.load_state_dict(torch.load('/content/drive/MyDrive/model_final.pt'))

corrects = 0
for batch in val_iter:
  x,y = batch.text.to(device), batch.label.to(device)
  y.data.sub_(1)
  hypothesis = model_new(x)
  corrects += (hypothesis.max(1)[1].view(y.size()).data == y.data).sum() 

print('Fianl Model Accuracy = ', corrects/len(val_iter.dataset)*100.0)

Fianl Model Accuracy =  tensor(56.8574, device='cuda:0')
